In [1]:
# Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

# You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

# They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

# Here is what the data looks like so far:

#     Description: Measurements of ship size, capacity, crew, and age for 158 cruise
#     ships.


#     Variables/Columns
#     Ship Name     1-20
#     Cruise Line   21-40
#     Age (as of 2013)   46-48
#     Tonnage (1000s of tons)   50-56
#     passengers (100s)   58-64
#     Length (100s of feet)  66-72
#     Cabins  (100s)   74-80
#     Passenger Density   82-88
#     Crew  (100s)   90-96
    
# It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

# Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [4]:
from pyspark.ml.regression import LinearRegression

In [5]:
data = spark.read.csv('/FileStore/tables/cruise_ship_info.csv',inferSchema=True,header=True)

In [6]:
data.describe().show()

+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
summary|Ship_name|Cruise_line| Age| Tonnage| passengers| length| cabins|passenger_density| crew|
+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
 count| 158| 158| 158| 158| 158| 158| 158| 158| 158|
 mean| Infinity| null|15.689873417721518| 71.28467088607599|18.45740506329114|8.130632911392404| 8.830000000000005|39.90094936708861|7.794177215189873|
 stddev| NaN| null| 7.615691058751413|37.229540025907866|9.677094775143416|1.793473548054825|4.4714172221480615| 8.63921711391542|3.503486564627034|
 min|Adventure| Azamara| 4| 2.329| 0.66| 2.79| 0.33| 17.7| 0.59|
 max|Zuiderdam| Windstar| 48| 220.0| 54.0| 11.82| 27.0| 71.43| 21.0|
+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+

In [7]:
data.printSchema()

root
-- Ship_name: string (nullable = true)
-- Cruise_line: string (nullable = true)
-- Age: integer (nullable = true)
-- Tonnage: double (nullable = true)
-- passengers: double (nullable = true)
-- length: double (nullable = true)
-- cabins: double (nullable = true)
-- passenger_density: double (nullable = true)
-- crew: double (nullable = true)

In [8]:
data.head(3)

Out[ 35 ]: 
[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55),
 Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55),
 Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7)]

In [9]:
#Since the company requested that we need to take account of cruise lines, I think we need to convert this column into a numerical one.
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol='Cruise_line', outputCol='cruise_cat')
indexed = indexer.fit(data).transform(data)
indexed.head(5)

Out[ 36 ]: 
[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0),
 Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0),
 Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7, cruise_cat=1.0),
 Row(Ship_name='Conquest', Cruise_line='Carnival', Age=11, Tonnage=110.0, passengers=29.74, length=9.53, cabins=14.88, passenger_density=36.99, crew=19.1, cruise_cat=1.0),
 Row(Ship_name='Destiny', Cruise_line='Carnival', Age=17, Tonnage=101.353, passengers=26.42, length=8.92, cabins=13.21, passenger_density=38.36, crew=10.0, cruise_cat=1.0)]

In [10]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [11]:
indexed.columns

Out[ 38 ]: 
['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'cruise_cat']

In [12]:
assembler = VectorAssembler(
  inputCols=['Age',
             'Tonnage',
             'passengers',
             'length',
             'cabins',
             'passenger_density',
             'cruise_cat'],
  outputCol='features')

In [13]:
output = assembler.transform(indexed)

In [14]:
output.select('features', 'crew').show()

+--------------------+----+
 features|crew|
+--------------------+----+
[6.0,30.276999999...|3.55|
[6.0,30.276999999...|3.55|
[26.0,47.262,14.8...| 6.7|
[11.0,110.0,29.74...|19.1|
[17.0,101.353,26....|10.0|
[22.0,70.367,20.5...| 9.2|
[15.0,70.367,20.5...| 9.2|
[23.0,70.367,20.5...| 9.2|
[19.0,70.367,20.5...| 9.2|
[6.0,110.23899999...|11.5|
[10.0,110.0,29.74...|11.6|
[28.0,46.052,14.5...| 6.6|
[18.0,70.367,20.5...| 9.2|
[17.0,70.367,20.5...| 9.2|
[11.0,86.0,21.24,...| 9.3|
[8.0,110.0,29.74,...|11.6|
[9.0,88.5,21.24,9...|10.3|
[15.0,70.367,20.5...| 9.2|
[12.0,88.5,21.24,...| 9.3|
[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows

In [15]:
final_data = output.select('features', 'crew')

In [16]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [17]:
lr = LinearRegression(labelCol='crew')

In [18]:
lrModel = lr.fit(train_data)

In [19]:
# Print the coefficients and intercept for linear regression
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [-0.00902045909628,0.0200360885998,-0.163078891387,0.372853134336,0.802360643693,-0.0186589025703,0.0612290691509] Intercept: -0.23412863102602977

In [20]:
test_results = lrModel.evaluate(test_data)

In [21]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))
print("R2: {}".format(test_results.r2))

RMSE: 1.2762112432418797
MSE: 1.6287151373769841
R2: 0.8865536897633275